# Bygg en klassificeringsmodell: Utsökta asiatiska och indiska rätter


## Introduktion till klassificering: Rensa, förbered och visualisera din data

I dessa fyra lektioner kommer du att utforska ett grundläggande fokus inom klassisk maskininlärning - *klassificering*. Vi kommer att gå igenom hur man använder olika klassificeringsalgoritmer med en dataset om alla de fantastiska köken i Asien och Indien. Hoppas du är hungrig!

<p >
   <img src="../../images/pinch.png"
   width="600"/>
   <figcaption>Fira pan-asiatiska kök i dessa lektioner! Bild av Jen Looper</figcaption>


<!--![Fira pan-asiatiska kök i dessa lektioner! Bild av Jen Looper](../../../../../../4-Classification/1-Introduction/solution/R/images/pinch.png)-->

Klassificering är en form av [övervakad inlärning](https://wikipedia.org/wiki/Supervised_learning) som har mycket gemensamt med regressionstekniker. Vid klassificering tränar du en modell för att förutsäga vilken `kategori` ett objekt tillhör. Om maskininlärning handlar om att förutsäga värden eller namn på saker med hjälp av datasets, så faller klassificering generellt sett in i två grupper: *binär klassificering* och *multiklassklassificering*.

Kom ihåg:

-   **Linjär regression** hjälpte dig att förutsäga relationer mellan variabler och göra exakta förutsägelser om var en ny datapunkt skulle hamna i förhållande till den linjen. Så du kunde förutsäga numeriska värden, som *vad priset på en pumpa skulle vara i september jämfört med december*, till exempel.

-   **Logistisk regression** hjälpte dig att upptäcka "binära kategorier": vid denna prisnivå, *är denna pumpa orange eller inte-orange*?

Klassificering använder olika algoritmer för att avgöra andra sätt att bestämma en datapunkts etikett eller klass. Låt oss arbeta med denna matlagningsdata för att se om vi, genom att observera en grupp ingredienser, kan avgöra dess ursprungskök.

### [**Quiz före föreläsningen**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/19/)

### **Introduktion**

Klassificering är en av de grundläggande aktiviteterna för forskare inom maskininlärning och dataanalytiker. Från grundläggande klassificering av ett binärt värde ("är detta e-postmeddelande skräppost eller inte?") till komplex bildklassificering och segmentering med hjälp av datorseende, är det alltid användbart att kunna sortera data i klasser och ställa frågor om den.

För att uttrycka processen på ett mer vetenskapligt sätt skapar din klassificeringsmetod en prediktiv modell som gör det möjligt att kartlägga relationen mellan indata och utdata.

<p >
   <img src="../../images/binary-multiclass.png"
   width="600"/>
   <figcaption>Binära vs. multiklassproblem för klassificeringsalgoritmer att hantera. Infografik av Jen Looper</figcaption>



Innan vi börjar processen med att rensa vår data, visualisera den och förbereda den för våra ML-uppgifter, låt oss lära oss lite om de olika sätt som maskininlärning kan användas för att klassificera data.

Härledd från [statistik](https://wikipedia.org/wiki/Statistical_classification), klassificering med klassisk maskininlärning använder egenskaper, såsom `rökare`, `vikt` och `ålder` för att avgöra *sannolikheten att utveckla X sjukdom*. Som en övervakad inlärningsteknik liknande de regressionsexempel du utförde tidigare, är din data märkt och ML-algoritmerna använder dessa etiketter för att klassificera och förutsäga klasser (eller 'egenskaper') i en dataset och tilldela dem till en grupp eller ett resultat.

✅ Ta en stund och föreställ dig en dataset om matlagning. Vad skulle en multiklassmodell kunna svara på? Vad skulle en binär modell kunna svara på? Vad händer om du ville avgöra om ett visst kök sannolikt använder bockhornsklöver? Vad händer om du ville se om du, med en present av en matkasse full av stjärnanis, kronärtskockor, blomkål och pepparrot, kunde skapa en typisk indisk maträtt?

### **Hej 'klassificerare'**

Frågan vi vill ställa om denna matlagningsdataset är faktiskt en **multiklassfråga**, eftersom vi har flera potentiella nationella kök att arbeta med. Givet en grupp ingredienser, vilken av dessa många klasser passar datan in i?

Tidymodels erbjuder flera olika algoritmer att använda för att klassificera data, beroende på vilken typ av problem du vill lösa. Under de kommande två lektionerna kommer du att lära dig om flera av dessa algoritmer.

#### **Förkunskaper**

För denna lektion behöver vi följande paket för att rensa, förbereda och visualisera vår data:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) är en [samling av R-paket](https://www.tidyverse.org/packages) designade för att göra dataanalys snabbare, enklare och roligare!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) är ett [ramverk av paket](https://www.tidymodels.org/packages/) för modellering och maskininlärning.

-   `DataExplorer`: [DataExplorer-paketet](https://cran.r-project.org/web/packages/DataExplorer/vignettes/dataexplorer-intro.html) är avsett att förenkla och automatisera EDA-processen och rapportgenerering.

-   `themis`: [themis-paketet](https://themis.tidymodels.org/) erbjuder extra receptsteg för att hantera obalanserad data.

Du kan installera dem med:

`install.packages(c("tidyverse", "tidymodels", "DataExplorer", "here"))`

Alternativt kontrollerar skriptet nedan om du har de paket som krävs för att slutföra denna modul och installerar dem åt dig om de saknas.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, DataExplorer, themis, here)

Vi kommer senare att ladda dessa fantastiska paket och göra dem tillgängliga i vår nuvarande R-session. (Detta är bara för illustration, `pacman::p_load()` har redan gjort det åt dig)


## Övning - rensa och balansera din data

Den första uppgiften, innan du börjar med detta projekt, är att rensa och **balansera** din data för att få bättre resultat.

Låt oss bekanta oss med datan!🕵️


In [ ]:
# Import data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# View the first 5 rows
df %>% 
  slice_head(n = 5)


Intressant! Det verkar som att den första kolumnen är en slags `id`-kolumn. Låt oss ta reda på lite mer information om data.


In [ ]:
# Basic information about the data
df %>%
  introduce()

# Visualize basic information above
df %>% 
  plot_intro(ggtheme = theme_light())

Från resultatet kan vi direkt se att vi har `2448` rader och `385` kolumner samt `0` saknade värden. Vi har också 1 diskret kolumn, *cuisine*.

## Övning - lära känna kökstyper

Nu börjar arbetet bli mer intressant. Låt oss utforska datafördelningen per kökstyp.


In [ ]:
# Count observations per cuisine
df %>% 
  count(cuisine) %>% 
  arrange(n)

# Plot the distribution
theme_set(theme_light())
df %>% 
  count(cuisine) %>% 
  ggplot(mapping = aes(x = n, y = reorder(cuisine, -n))) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("cuisine")

Det finns ett begränsat antal kök, men fördelningen av data är ojämn. Du kan fixa det! Utforska lite mer innan du gör det.

Låt oss nu tilldela varje kök till sin egen tibble och ta reda på hur mycket data som finns tillgängligt (rader, kolumner) per kök.

> En [tibble](https://tibble.tidyverse.org/) är en modern dataram.

<p >
   <img src="../../images/dplyr_filter.jpg"
   width="600"/>
   <figcaption>Konstverk av @allison_horst</figcaption>


In [ ]:
# Create individual tibble for the cuisines
thai_df <- df %>% 
  filter(cuisine == "thai")
japanese_df <- df %>% 
  filter(cuisine == "japanese")
chinese_df <- df %>% 
  filter(cuisine == "chinese")
indian_df <- df %>% 
  filter(cuisine == "indian")
korean_df <- df %>% 
  filter(cuisine == "korean")


# Find out how much data is available per cuisine
cat(" thai df:", dim(thai_df), "\n",
    "japanese df:", dim(japanese_df), "\n",
    "chinese_df:", dim(chinese_df), "\n",
    "indian_df:", dim(indian_df), "\n",
    "korean_df:", dim(korean_df))

## **Övning - Upptäcka toppingredienser per kök med hjälp av dplyr**

Nu kan du gräva djupare i datan och ta reda på vilka som är de typiska ingredienserna för varje kök. Du bör rensa bort återkommande data som skapar förvirring mellan köken, så låt oss lära oss mer om detta problem.

Skapa en funktion `create_ingredient()` i R som returnerar en ingrediens-dataram. Denna funktion börjar med att ta bort en oanvändbar kolumn och sortera ingredienser baserat på deras antal.

Den grundläggande strukturen för en funktion i R är:

`myFunction <- function(arglist){`

**`...`**

**`return`**`(value)`

`}`

En enkel introduktion till R-funktioner finns [här](https://skirmer.github.io/presentations/functions_with_r.html#1).

Låt oss sätta igång! Vi kommer att använda [dplyr-verb](https://dplyr.tidyverse.org/) som vi har lärt oss i våra tidigare lektioner. Som en sammanfattning:

-   `dplyr::select()`: hjälper dig att välja vilka **kolumner** du vill behålla eller utesluta.

-   `dplyr::pivot_longer()`: hjälper dig att "förlänga" data, vilket ökar antalet rader och minskar antalet kolumner.

-   `dplyr::group_by()` och `dplyr::summarise()`: hjälper dig att hitta sammanfattande statistik för olika grupper och presentera dem i en snygg tabell.

-   `dplyr::filter()`: skapar en delmängd av datan som endast innehåller rader som uppfyller dina villkor.

-   `dplyr::mutate()`: hjälper dig att skapa eller ändra kolumner.

Kolla in denna [*konst*-fyllda learnr-tutorial](https://allisonhorst.shinyapps.io/dplyr-learnr/#section-welcome) av Allison Horst, som introducerar några användbara datahanteringsfunktioner i dplyr *(en del av Tidyverse)*.


In [ ]:
# Creates a functions that returns the top ingredients by class

create_ingredient <- function(df){
  
  # Drop the id column which is the first colum
  ingredient_df = df %>% select(-1) %>% 
  # Transpose data to a long format
    pivot_longer(!cuisine, names_to = "ingredients", values_to = "count") %>% 
  # Find the top most ingredients for a particular cuisine
    group_by(ingredients) %>% 
    summarise(n_instances = sum(count)) %>% 
    filter(n_instances != 0) %>% 
  # Arrange by descending order
    arrange(desc(n_instances)) %>% 
    mutate(ingredients = factor(ingredients) %>% fct_inorder())
  
  
  return(ingredient_df)
} # End of function

Nu kan vi använda funktionen för att få en uppfattning om de tio mest populära ingredienserna per kök. Låt oss testa den med `thai_df`.


In [ ]:
# Call create_ingredient and display popular ingredients
thai_ingredient_df <- create_ingredient(df = thai_df)

thai_ingredient_df %>% 
  slice_head(n = 10)

I föregående avsnitt använde vi `geom_col()`, låt oss se hur du också kan använda `geom_bar` för att skapa stapeldiagram. Använd `?geom_bar` för vidare läsning.


In [ ]:
# Make a bar chart for popular thai cuisines
thai_ingredient_df %>% 
  slice_head(n = 10) %>% 
  ggplot(aes(x = n_instances, y = ingredients)) +
  geom_bar(stat = "identity", width = 0.5, fill = "steelblue") +
  xlab("") + ylab("")

Låt oss göra samma sak för de japanska uppgifterna


In [ ]:
# Get popular ingredients for Japanese cuisines and make bar chart
create_ingredient(df = japanese_df) %>% 
  slice_head(n = 10) %>%
  ggplot(aes(x = n_instances, y = ingredients)) +
  geom_bar(stat = "identity", width = 0.5, fill = "darkorange", alpha = 0.8) +
  xlab("") + ylab("")


Vad sägs om de kinesiska köken?


In [ ]:
# Get popular ingredients for Chinese cuisines and make bar chart
create_ingredient(df = chinese_df) %>% 
  slice_head(n = 10) %>%
  ggplot(aes(x = n_instances, y = ingredients)) +
  geom_bar(stat = "identity", width = 0.5, fill = "cyan4", alpha = 0.8) +
  xlab("") + ylab("")

In [ ]:
# Get popular ingredients for Indian cuisines and make bar chart
create_ingredient(df = indian_df) %>% 
  slice_head(n = 10) %>%
  ggplot(aes(x = n_instances, y = ingredients)) +
  geom_bar(stat = "identity", width = 0.5, fill = "#041E42FF", alpha = 0.8) +
  xlab("") + ylab("")

Slutligen, plotta de koreanska ingredienserna.


In [ ]:
# Get popular ingredients for Korean cuisines and make bar chart
create_ingredient(df = korean_df) %>% 
  slice_head(n = 10) %>%
  ggplot(aes(x = n_instances, y = ingredients)) +
  geom_bar(stat = "identity", width = 0.5, fill = "#852419FF", alpha = 0.8) +
  xlab("") + ylab("")

Från datavisualiseringarna kan vi nu ta bort de vanligaste ingredienserna som skapar förvirring mellan olika kök, med hjälp av `dplyr::select()`.

Alla älskar ris, vitlök och ingefära!


In [ ]:
# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger))

# Display new data set
df_select %>% 
  slice_head(n = 5)

## Förbehandling av data med recept 👩‍🍳👨‍🍳 - Hantera obalanserad data ⚖️

<p >
   <img src="../../images/recipes.png"
   width="600"/>
   <figcaption>Konstverk av @allison_horst</figcaption>

Eftersom den här lektionen handlar om kök, måste vi sätta `recept` i rätt sammanhang.

Tidymodels erbjuder ännu ett smidigt paket: `recipes` - ett paket för förbehandling av data.


Låt oss titta på fördelningen av våra kök igen.


In [ ]:
# Distribution of cuisines
old_label_count <- df_select %>% 
  count(cuisine) %>% 
  arrange(desc(n))

old_label_count

Som du kan se, finns det en ganska ojämn fördelning i antalet kök. Koreanska kök är nästan tre gånger fler än thailändska kök. Obalanserad data har ofta negativa effekter på modellens prestanda. Tänk på en binär klassificering. Om majoriteten av din data tillhör en klass, kommer en ML-modell att förutsäga den klassen oftare, bara för att det finns mer data för den. Att balansera data tar bort snedvridningar och hjälper till att eliminera denna obalans. Många modeller presterar bäst när antalet observationer är lika och har därför svårt med obalanserad data.

Det finns huvudsakligen två sätt att hantera obalanserade datasätt:

-   lägga till observationer till minoritetsklassen: `Över-sampling`, t.ex. med hjälp av en SMOTE-algoritm

-   ta bort observationer från majoritetsklassen: `Under-sampling`

Låt oss nu demonstrera hur man hanterar obalanserade datasätt med hjälp av ett `recept`. Ett recept kan ses som en ritning som beskriver vilka steg som ska tillämpas på ett datasätt för att göra det redo för dataanalys.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing data
cuisines_recipe <- recipe(cuisine ~ ., data = df_select) %>% 
  step_smote(cuisine)

cuisines_recipe

Låt oss gå igenom våra förbehandlingssteg.

-   Anropet till `recipe()` med en formel talar om för receptet vilka *roller* variablerna har, med hjälp av `df_select`-data som referens. Till exempel har kolumnen `cuisine` tilldelats rollen `outcome`, medan resten av kolumnerna har tilldelats rollen `predictor`.

-   [`step_smote(cuisine)`](https://themis.tidymodels.org/reference/step_smote.html) skapar en *specifikation* för ett receptsteg som syntetiskt genererar nya exempel för minoritetsklassen med hjälp av närmaste grannar till dessa fall.

Nu, om vi vill se den förbehandlade datan, måste vi [**`prep()`**](https://recipes.tidymodels.org/reference/prep.html) och [**`bake()`**](https://recipes.tidymodels.org/reference/bake.html) vårt recept.

`prep()`: uppskattar de nödvändiga parametrarna från en träningsuppsättning som senare kan tillämpas på andra dataset.

`bake()`: tar ett förberett recept och tillämpar operationerna på valfritt dataset.


In [ ]:
# Prep and bake the recipe
preprocessed_df <- cuisines_recipe %>% 
  prep() %>% 
  bake(new_data = NULL) %>% 
  relocate(cuisine)

# Display data
preprocessed_df %>% 
  slice_head(n = 5)

# Quick summary stats
preprocessed_df %>% 
  introduce()

Låt oss nu kontrollera fördelningen av våra kök och jämföra dem med den obalanserade datan.


In [ ]:
# Distribution of cuisines
new_label_count <- preprocessed_df %>% 
  count(cuisine) %>% 
  arrange(desc(n))

list(new_label_count = new_label_count,
     old_label_count = old_label_count)

Mums! Datan är ren, balanserad och väldigt läcker 😋!

> Vanligtvis används ett recept som en förprocessor för modellering där det definierar vilka steg som ska tillämpas på en dataset för att göra den redo för modellering. I sådana fall används vanligtvis en `workflow()` (som vi redan har sett i våra tidigare lektioner) istället för att manuellt uppskatta ett recept.
>
> Därför behöver du vanligtvis inte **`prep()`** och **`bake()`** recept när du använder tidymodels, men de är användbara funktioner att ha i din verktygslåda för att bekräfta att recepten gör det du förväntar dig, som i vårt fall.
>
> När du **`bake()`** ett förberett recept med **`new_data = NULL`**, får du tillbaka den data som du angav när du definierade receptet, men som har genomgått förbearbetningsstegen.

Låt oss nu spara en kopia av denna data för användning i framtida lektioner:


In [ ]:
# Save preprocessed data
write_csv(preprocessed_df, "../../../data/cleaned_cuisines_R.csv")

Denna nya CSV-fil finns nu i rotmappen för data.

**🚀Utmaning**

Detta kursmaterial innehåller flera intressanta dataset. Gå igenom `data`-mapparna och se om någon innehåller dataset som skulle passa för binär eller multi-klassklassificering? Vilka frågor skulle du ställa till detta dataset?

## [**Quiz efter föreläsningen**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/20/)

## **Granskning & Självstudier**

-   Kolla in [paketet themis](https://github.com/tidymodels/themis). Vilka andra tekniker kan vi använda för att hantera obalanserad data?

-   Tidy models [referenswebbplats](https://www.tidymodels.org/start/).

-   H. Wickham och G. Grolemund, [*R för Data Science: Visualisera, Modellera, Transformera, Städa och Importera Data*](https://r4ds.had.co.nz/).

#### TACK TILL:

[`Allison Horst`](https://twitter.com/allison_horst/) för att ha skapat de fantastiska illustrationerna som gör R mer välkomnande och engagerande. Hitta fler illustrationer i hennes [galleri](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) och [Jen Looper](https://www.twitter.com/jenlooper) för att ha skapat den ursprungliga Python-versionen av denna modul ♥️

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="600"/>
   <figcaption>Konstverk av @allison_horst</figcaption>



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, vänligen notera att automatiska översättningar kan innehålla fel eller felaktigheter. Det ursprungliga dokumentet på dess originalspråk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
